In [1]:
import pandas as pd

In [4]:
transl_path = "/home/craig.car/repos/chiron/align_texts_project/data/translations_repositories.csv"

In [138]:
transl_df = pd.read_csv(transl_path)
transl_df = transl_df.rename(columns={"CTS-URN/s":"cts-urns_grp", "Original File Name/Location":"Original_File_Name"})

In [139]:
transl_df.loc[0,"cts-urns_grp"]

'urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:cts:greekLit:tlg0085.tlg002.ogl-eng1 urn:cts:greekLit:tlg0085.tlg003.ogl-eng1 urn:cts:greekLit:tlg0085.tlg004.ogl-eng1 urn:cts:greekLit:tlg0085.tlg005.ogl-eng1 urn:cts:greekLit:tlg0085.tlg006.ogl-eng1 urn:cts:greekLit:tlg0085.tlg007.ogl-eng1'

In [140]:
transl_df.loc[0,"cts-urns_grp"].split()

['urn:cts:greekLit:tlg0085.tlg001.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg002.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg003.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg004.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg005.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg006.ogl-eng1',
 'urn:cts:greekLit:tlg0085.tlg007.ogl-eng1']

In [141]:
transl_df["cts-urns_uniques"] = transl_df["cts-urns_grp"].str.split()

In [142]:
transl_df = transl_df.explode("cts-urns_uniques", ignore_index=True)

In [143]:
transl_df["ctsurns"] = transl_df["cts-urns_uniques"].str.rsplit(".",1).str[0]

In [144]:
transl_df["filename"] = transl_df["Original_File_Name"].str.split(".").str[0]

In [145]:
transl_df.head()

,Author Name,WORK IDs,cts-urns_grp,# of translations,Edition Title,Editor; Translator,Publisher,Date,Original_File_Name,Addiional File location/locations,...,Translation Language,Worldcat Link,Work/s Split Up/Renamed in Branch,New File Name/s,Notes,TOC,Additional TOC,cts-urns_uniques,ctsurns,filename
0,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,English,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1,urn:cts:greekLit:tlg0085.tlg001,aeschylus_1886
1,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,English,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg002.ogl-eng1,urn:cts:greekLit:tlg0085.tlg002,aeschylus_1886
2,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,English,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg003.ogl-eng1,urn:cts:greekLit:tlg0085.tlg003,aeschylus_1886
3,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,English,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg004.ogl-eng1,urn:cts:greekLit:tlg0085.tlg004,aeschylus_1886
4,Aeschylus,0085.001 0085.002 0085.003 0085.004 0085.005 0...,urn:cts:greekLit:tlg0085.tlg001.ogl-eng1 urn:c...,7.0,The Dramas of Aeschylus,"Swanwick, Anna, 1831-1899",Bell,1886,aeschylus_1886.xml,NaN,...,English,NaN,no,NaN,NaN,NaN,NaN,urn:cts:greekLit:tlg0085.tlg005.ogl-eng1,urn:cts:greekLit:tlg0085.tlg005,aeschylus_1886


In [156]:
transl_df["ctsurns"].duplicated().sum()

796

In [158]:
keys = transl_df["ctsurns"].unique()

In [166]:
lookup_table = {}
for ctsurn in keys:
    lookup_table[ctsurn] = list(transl_df.loc[transl_df["ctsurns"] == ctsurn, "filename"])

In [172]:
# spot check dict values
transl_df.loc[transl_df["ctsurns"] == "urn:cts:greekLit:tlg0085.tlg004", "filename"]

3     aeschylus_1886
10    aeschylus_1888
17    aeschylus_1909
27    aeschylus_1872
37    aeschylus_1868
Name: filename, dtype: object

In [167]:
lookup_table

{'urn:cts:greekLit:tlg0085.tlg001': ['aeschylus_1886',
  'aeschylus_1888',
  'aeschylus_1909',
  'aeschylus_1872',
  'aeschylus_1868'],
 'urn:cts:greekLit:tlg0085.tlg002': ['aeschylus_1886',
  'aeschylus_1888',
  'aeschylus_1909',
  'aeschylus_1872',
  'aeschylus_1868'],
 'urn:cts:greekLit:tlg0085.tlg003': ['aeschylus_1886',
  'aeschylus_1888',
  'aeschylus_1909',
  'aeschylus_1872',
  'aeschylus_1868',
  'greekDramas_1912'],
 'urn:cts:greekLit:tlg0085.tlg004': ['aeschylus_1886',
  'aeschylus_1888',
  'aeschylus_1909',
  'aeschylus_1872',
  'aeschylus_1868'],
 'urn:cts:greekLit:tlg0085.tlg005': ['aeschylus_1886',
  'aeschylus_1888',
  'aeschylus_1909',
  'aeschylus_1863',
  'aeschylus_1872',
  'Aeschylus_1901',
  'aeschylus_1868',
  'greekDramas_1912'],
 'urn:cts:greekLit:tlg0085.tlg006': ['aeschylus_1886',
  'aeschylus_1888',
  'aeschylus_1909',
  'aeschylus_1863',
  'aeschylus_1872',
  'Aeschylus_1901',
  'aeschylus_1868'],
 'urn:cts:greekLit:tlg0085.tlg007': ['aeschylus_1886',
  'ae